# Clustering

## K-Means

The KMeans algorithm clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum-of-squares. This algorithm requires the number of clusters to be specified.

The k-means algorithm divides a set of $N$ samples $X$ into $K$ disjoint clusters $C$, each described by the mean $\mu_j$ of the samples in the cluster. The means are commonly called the cluster “centroids”. The K-means algorithm aims to choose centroids that minimise the inertia, or within-cluster sum-of-squares criterion:
$$\sum_{i=0}^{n}\min_{\mu_j \in C}(||x_i - \mu_j||^2)$$

### The Algorithm

K-means is often referred to as Lloyd’s algorithm. In basic terms, the algorithm has three steps. The first step chooses the initial centroids, with the most basic method being to choose $k$ samples from the dataset $X$. After initialization, K-means consists of looping between the two other steps. The first step assigns each sample to its nearest centroid. The second step creates new centroids by taking the mean value of all of the samples assigned to each previous centroid. The difference between the old and the new centroids are computed and the algorithm repeats these last two steps until this value is less than a threshold. In other words, it repeats until the centroids do not move significantly.

Given enough time, K-means will always converge, however this may be to a local minimum. This is highly dependent on the initialization of the centroids. As a result, the computation is often done several times, with different initializations of the centroids. One method to help address this issue is the k-means++ initialization scheme, which has been implemented in scikit-learn (use the `init='k-means++'` parameter). This initializes the centroids to be (generally) distant from each other, leading to probably better results than random initialization, as shown in the reference.

The algorithm supports sample weights, which can be given by a parameter sample_weight. This allows to assign more weight to some samples when computing cluster centers and values of inertia. For example, assigning a weight of 2 to a sample is equivalent to adding a duplicate of that sample to the dataset $X$.

See the [paper](http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf) for more details.

### Mini Batch K-Means

The `MiniBatchKMeans` is a variant of the `KMeans` algorithm which uses mini-batches to reduce the computation time, while still attempting to optimise the same objective function. Mini-batches are subsets of the input data, randomly sampled in each training iteration. These mini-batches drastically reduce the amount of computation required to converge to a local solution. In contrast to other algorithms that reduce the convergence time of k-means, mini-batch k-means produces results that are generally only slightly worse than the standard algorithm.

The algorithm iterates between two major steps, similar to vanilla k-means. In the first step, $b$ samples are drawn randomly from the dataset, to form a mini-batch. These are then assigned to the nearest centroid. In the second step, the centroids are updated. In contrast to k-means, this is done on a per-sample basis. For each sample in the mini-batch, the assigned centroid is updated by taking the streaming average of the sample and all previous samples assigned to that centroid. This has the effect of decreasing the rate of change for a centroid over time. These steps are performed until convergence or a predetermined number of iterations is reached.


>`MiniBatchKMeans` converges faster than `KMeans`, but the quality of the results is reduced.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_mini_batch_kmeans_001.png)

## Affinity Propagation

`AffinityPropagation` creates clusters by sending messages between pairs of samples until convergence. A dataset is then described using a small number of exemplars, which are identified as those most representative of other samples. The messages sent between pairs represent the suitability for one sample to be the exemplar of the other, which is updated in response to the values from other pairs. This updating happens iteratively until convergence, at which point the final exemplars are chosen, and hence the final clustering is given.

Affinity Propagation can be interesting as it chooses the number of clusters based on the data provided. For this purpose, the two important parameters are the *preference*, which controls how many exemplars are used, and the *damping factor* which damps the responsibility and availability messages to avoid numerical oscillations when updating these messages.

The main drawback of Affinity Propagation is its complexity. The algorithm has a time complexity of the order $O(N^2 T)$, where $N$ is the number of samples and $T$ is the number of iterations until convergence. Further, the memory complexity is of the order $O(N^2)$ if a dense similarity matrix is used, but reducible if a sparse similarity matrix is used. This makes Affinity Propagation most appropriate for small to medium sized datasets.

## Hierarchical clustering

Hierarchical clustering is a general family of clustering algorithms that build nested clusters by merging or splitting them successively. This hierarchy of clusters is represented as a tree (or dendrogram). The root of the tree is the unique cluster that gathers all the samples, the leaves being the clusters with only one sample.

The `AgglomerativeClustering` object performs a hierarchical clustering using a bottom up approach: each observation starts in its own cluster, and clusters are successively merged together. The linkage criteria determines the metric used for the merge strategy:
<ul>
<li><p><strong>Ward</strong> minimizes the sum of squared differences within all clusters. It is a
variance-minimizing approach and in this sense is similar to the k-means
objective function but tackled with an agglomerative hierarchical
approach.</p></li>
<li><p><strong>Maximum</strong> or <strong>complete linkage</strong> minimizes the maximum distance between
observations of pairs of clusters.</p></li>
<li><p><strong>Average linkage</strong> minimizes the average of the distances between all
observations of pairs of clusters.</p></li>
<li><p><strong>Single linkage</strong> minimizes the distance between the closest
observations of pairs of clusters.</p></li>
</ul>

### Different linkage type: Ward, complete, average, and single linkage

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_linkage_comparison_001.png)

Agglomerative cluster has a “rich get richer” behavior that leads to uneven cluster sizes. In this regard, single linkage is the worst strategy, and Ward gives the most regular sizes. However, the affinity (or distance used in clustering) cannot be varied with Ward, thus for non Euclidean metrics, average linkage is a good alternative. Single linkage, while not robust to noisy data, can be computed very efficiently and can therefore be useful to provide hierarchical clustering of larger datasets. Single linkage can also perform well on non-globular data.

### Visualization of cluster hierarchy

It’s possible to visualize the tree representing the hierarchical merging of clusters as a dendrogram. Visual inspection can often be useful for understanding the structure of the data, though more so in the case of small sample sizes.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_dendrogram_001.png)

### Adding connectivity constraints

An interesting aspect of `AgglomerativeClustering` is that connectivity constraints can be added to this algorithm (only adjacent clusters can be merged together), through a connectivity matrix that defines for each sample the neighboring samples following a given structure of the data. For instance, in the swiss-roll example below, the connectivity constraints forbid the merging of points that are not adjacent on the swiss roll, and thus avoid forming clusters that extend across overlapping folds of the roll.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_ward_structured_vs_unstructured_001.png)
![](https://scikit-learn.org/stable/_images/sphx_glr_plot_ward_structured_vs_unstructured_002.png)

### Varying the metric

<p>Single, average and complete linkage can be used with a variety of distances (or
affinities), in particular Euclidean distance (<em>l2</em>), Manhattan distance
(or Cityblock, or <em>l1</em>), cosine distance, or any precomputed affinity
matrix.</p>

<ul class="simple">
<li><p><em>l1</em> distance is often good for sparse features, or sparse noise: i.e.
many of the features are zero, as in text mining using occurrences of
rare words.</p></li>
<li><p><em>cosine</em> distance is interesting because it is invariant to global
scalings of the signal.</p></li>
</ul>

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_005.png)
![](https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_006.png)
![](https://scikit-learn.org/stable/_images/sphx_glr_plot_agglomerative_clustering_metrics_007.png)

## DBSCAN

The `DBSCAN` algorithm views clusters as areas of high density separated by areas of low density. Due to this rather generic view, clusters found by DBSCAN can be any shape, as opposed to k-means which assumes that clusters are convex shaped. The central component to the DBSCAN is the concept of core samples, which are samples that are in areas of high density. A cluster is therefore a set of core samples, each close to each other (measured by some distance measure) and a set of non-core samples that are close to a core sample (but are not themselves core samples). There are two parameters to the algorithm, `min_samples` and `eps`, which define formally what we mean when we say dense. Higher `min_samples` or lower `eps` indicate higher density necessary to form a cluster.

More formally, we define a core sample as being a sample in the dataset such that there exist `min_samples` other samples within a distance of `eps`, which are defined as neighbors of the core sample. This tells us that the core sample is in a dense area of the vector space. A cluster is a set of core samples that can be built by recursively taking a core sample, finding all of its neighbors that are core samples, finding all of their neighbors that are core samples, and so on. A cluster also has a set of non-core samples, which are samples that are neighbors of a core sample in the cluster but are not themselves core samples. Intuitively, these samples are on the fringes of a cluster.

Any core sample is part of a cluster, by definition. Any sample that is not a core sample, and is at least `eps` in distance from any core sample, is considered an outlier by the algorithm.

While the parameter `min_samples` primarily controls how tolerant the algorithm is towards noise (on noisy and large data sets it may be desirable to increase this parameter), the parameter `eps` is crucial to choose appropriately for the data set and distance function and usually cannot be left at the default value. It controls the local neighborhood of the points. When chosen too small, most data will not be clustered at all (and labeled as `-1 `for “noise”). When chosen too large, it causes close clusters to be merged into one cluster, and eventually the entire data set to be returned as a single cluster. 

### Implementation

The DBSCAN algorithm is deterministic, always generating the same clusters when given the same data in the same order. However, the results can differ when data is provided in a different order. First, even though the core samples will always be assigned to the same clusters, the labels of those clusters will depend on the order in which those samples are encountered in the data. Second and more importantly, the clusters to which non-core samples are assigned can differ depending on the data order. This would happen when a non-core sample has a distance lower than `eps` to two core samples in different clusters. By the triangular inequality, those two core samples must be more distant than `eps` from each other, or they would be in the same cluster. The non-core sample is assigned to whichever cluster is generated first in a pass through the data, and so the results will depend on the data ordering.

The current implementation uses ball trees and kd-trees to determine the neighborhood of points, which avoids calculating the full distance matrix.

## Clustering performance evaluation

### Rand index

Given the knowledge of the ground truth class assignments `labels_true` and our clustering algorithm assignments of the same samples `labels_pred`, the (adjusted or unadjusted) Rand index is a function that measures the similarity of the two assignments, ignoring permutations:

In [1]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]
metrics.rand_score(labels_true, labels_pred)

0.6666666666666666

The Rand index does not ensure to obtain a value close to 0.0 for a random labelling. The adjusted Rand index corrects for chance and will give such a baseline.

As with all clustering metrics, one can permute 0 and 1 in the predicted labels, rename 2 to 3, and get the same score:

In [2]:
labels_pred = [1, 1, 0, 0, 3, 3]
print(metrics.rand_score(labels_true, labels_pred))

metrics.adjusted_rand_score(labels_true, labels_pred)

0.6666666666666666


0.24242424242424243

Furthermore, both `rand_score` and `adjusted_rand_score` are symmetric: swapping the argument does not change the scores.

### Mutual Information based scores

Given the knowledge of the ground truth class assignments `labels_true` and our clustering algorithm assignments of the same samples `labels_pred`, the Mutual Information is a function that measures the agreement of the two assignments, ignoring permutations. Two different normalized versions of this measure are available, Normalized Mutual Information (NMI) and Adjusted Mutual Information (AMI). NMI is often used in the literature, while AMI was proposed more recently and is normalized against chance:

In [3]:
from sklearn import metrics
labels_true = [0, 0, 0, 1, 1, 1]
labels_pred = [0, 0, 1, 1, 2, 2]

metrics.adjusted_mutual_info_score(labels_true, labels_pred) 

0.2987924581708901

In [4]:
labels_pred = [1, 1, 0, 0, 3, 3]
metrics.adjusted_mutual_info_score(labels_true, labels_pred)  


0.2987924581708901

### Silhouette Coefficient

<p>If the ground truth labels are not known, evaluation must be performed using
the model itself. The Silhouette Coefficient
is an example of such an evaluation, where a
higher Silhouette Coefficient score relates to a model with better defined
clusters. The Silhouette Coefficient is defined for each sample and is composed
of two scores:</p>
<ul class="simple">
<li><p><strong>a</strong>: The mean distance between a sample and all other points in the same
class.</p></li>
<li><p><strong>b</strong>: The mean distance between a sample and all other points in the <em>next
nearest cluster</em>.</p></li>
</ul>

The Silhouette Coefficient s for a single sample is then given as:
$$s = \frac{b - a}{max(a, b)}$$
The Silhouette Coefficient for a set of samples is given as the mean of the Silhouette Coefficient for each sample.

In [5]:
from sklearn import metrics
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)
import numpy as np
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans_model.labels_
metrics.silhouette_score(X, labels, metric='euclidean')

0.5528190123564095

# Algorithms

## `KMeans`

### Parameters of `KMeans`

Here are some of the parameters of the `KMeans` algorithm:
* **n_clusters**: int, default=8
  
    The number of clusters to form as well as the number of centroids to generate.
* **init**: {‘k-means++’, ‘random’}, callable or array-like of shape (n_clusters, n_features), default=’k-means++’
  
    Method for initialization, defaults to ‘k-means++’:
    * ‘k-means++’: Selects initial cluster centers for k-mean clustering in a smart way to speed up convergence. See section Notes for more details.
    * ‘random’: Initialize the centroids randomly.
    * If an array is passed, it should be of shape (n_clusters, n_features) and gives the initial centers.
    * If a callable is passed, it should take arguments X, n_clusters and a random state and return an initialization.
* **n_init**: int, default=10
  
    Number of time the k-means algorithm will be run with different centroid seeds. The final results will be the best output of n_init consecutive runs in terms of inertia.
* **max_iter**: int, default=300
    
    Maximum number of iterations of the k-means algorithm for a single run.
* **tol**: float, default=1e-4
    
    Relative tolerance with regards to inertia to declare convergence.
* **algorithm**: {“lloyd”, “elkan”, “auto”, “full”}, default=”lloyd”
    
    K-means algorithm to use. The classical EM-style algorithm is "lloyd". The "elkan" variation can be more efficient on some datasets with well-defined clusters, by using the triangle inequality. However it’s more memory intensive due to the allocation of an extra array of shape (n_samples, n_clusters).

    "auto" and "full" are deprecated and they will be removed in Scikit-Learn 1.3. They are both aliases for "lloyd"

### Attributes of `KMeans`

* **cluster_centers_**: array, [n_clusters, n_features]
  
    Coordinates of cluster centers
* **labels_**: array, [n_samples]
      
     Labels of each point
* **inertia_**: float
  
    The final value of the inertia criterion associated with the chosen clustering.

## `AffinityPropagation`

### Parameters of `AffinityPropagation`

* **damping**: float, default=0.5
  
    Damping factor in the range [0.5, 1.0) is the extent to which the current value is maintained relative to incoming values (weighted 1 - damping). This in order to avoid numerical oscillations when updating these values.
* **convergence_iter**: int, default=15
  
    Number of iterations with no change in the number of estimated clusters to consider the algorithm converged.
* **max_iter**: int, default=200
    
    Maximum number of iterations to perform.
* **preference**: array-like of shape (n_samples,) or float, default=None
  
    Preferences for each point - points with larger values of preferences are more likely to be chosen as exemplars. The number of exemplars, ie of clusters, is influenced by the input preferences value. If the preferences are not passed as arguments, they will be set to the median of the input similarities.
* **affinity**: {“euclidean”, “precomputed”}, default=”euclidean”
  
    Which affinity to use. At the moment ‘precomputed’ and euclidean are supported. ‘euclidean’ uses the negative squared euclidean distance between points.

### Attributes of `AffinityPropagation`

* **cluster_centers_indices_**: ndarray of shape (n_clusters,)

    Indices of cluster centers.
* **cluster_centers_**: ndarray of shape (n_clusters, n_features)
  
    Coordinates of cluster centers.
* **labels_**: ndarray of shape (n_samples,)
      
     Labels of each point
* **affinity_matrix_**: ndarray of shape (n_samples, n_samples)
  
    Stores the affinity matrix used to generate the clustering.

## `AgglomerativeClustering`

### Parameters of `AgglomerativeClustering`

<dd><dl>
<dt><strong>n_clusters</strong>: <span>int or None, default=2</span></dt><dd><p>The number of clusters to find. It must be <code><span>None</span></code> if
<code><span>distance_threshold</span></code> is not <code><span>None</span></code>.</p>
</dd>
<dt><strong>affinity</strong>: <span>str or callable, default=’euclidean’</span></dt><dd><p>Metric used to compute the linkage. Can be “euclidean”, “l1”, “l2”,
“manhattan”, “cosine”, or “precomputed”.
If linkage is “ward”, only “euclidean” is accepted.
If “precomputed”, a distance matrix (instead of a similarity matrix)
is needed as input for the fit method.</p>
</dd>
<dt><strong>memory</strong>: <span>str or object with the joblib.Memory interface, default=None</span></dt><dd><p>Used to cache the output of the computation of the tree.
By default, no caching is done. If a string is given, it is the
path to the caching directory.</p>
</dd>
<dt><strong>connectivity</strong>: <span>array-like or callable, default=None</span></dt><dd><p>Connectivity matrix. Defines for each sample the neighboring
samples following a given structure of the data.
This can be a connectivity matrix itself or a callable that transforms
the data into a connectivity matrix, such as derived from
<code><span>kneighbors_graph</span></code>. Default is <code><span>None</span></code>, i.e, the
hierarchical clustering algorithm is unstructured.</p>
</dd>
<dt><strong>compute_full_tree</strong>: <span>‘auto’ or bool, default=’auto’</span></dt><dd><p>Stop early the construction of the tree at <code><span>n_clusters</span></code>. This is
useful to decrease computation time if the number of clusters is not
small compared to the number of samples. This option is useful only
when specifying a connectivity matrix. Note also that when varying the
number of clusters and using caching, it may be advantageous to compute
the full tree. It must be <code><span>True</span></code> if <code><span>distance_threshold</span></code> is not
<code><span>None</span></code>. By default <code><span>compute_full_tree</span></code> is “auto”, which is equivalent
to <code><span>True</span></code> when <code><span>distance_threshold</span></code> is not <code><span>None</span></code> or that <code><span>n_clusters</span></code>
is inferior to the maximum between 100 or <code><span>0.02</span> <span>*</span> <span>n_samples</span></code>.
Otherwise, “auto” is equivalent to <code><span>False</span></code>.</p>
</dd>
<dt><strong>linkage</strong>: <span>{‘ward’, ‘complete’, ‘average’, ‘single’}, default=’ward’</span></dt><dd><p>Which linkage criterion to use. The linkage criterion determines which
distance to use between sets of observation. The algorithm will merge
the pairs of cluster that minimize this criterion.</p>
<ul class="simple">
<li><p>‘ward’ minimizes the variance of the clusters being merged.</p></li>
<li><p>‘average’ uses the average of the distances of each observation of
the two sets.</p></li>
<li><p>‘complete’ or ‘maximum’ linkage uses the maximum distances between
all observations of the two sets.</p></li>
<li><p>‘single’ uses the minimum of the distances between all observations
of the two sets.</p></li>
</ul>
</dd>
<dt><strong>distance_threshold</strong>: <span>float, default=None</span></dt><dd><p>The linkage distance threshold above which, clusters will not be
merged. If not <code><span>None</span></code>, <code><span>n_clusters</span></code> must be <code><span>None</span></code> and
<code><span>compute_full_tree</span></code> must be <code><span>True</span></code>.</p>
</dd>
<dt><strong>compute_distances</strong>: <span>bool, default=False</span></dt><dd><p>Computes distances between clusters even if <code><span>distance_threshold</span></code> is not
used. This can be used to make dendrogram visualization, but introduces
a computational and memory overhead.</p>
</dd>
</dl>
</dd>

### Attributes of `AgglomerativeClustering`

* **n_clusters_**: int
  
    The number of clusters found by the algorithm.
* **labels_**: ndarray of shape (n_samples,)
  
    Labels of each point
* **n_leaves_**: int
  
    Number of leaves in the tree.
* **n_connected_components_**: int
  
    The estimated number of connected components in the graph.
* **children_**: ndarray of shape (n_nodes-1, 2)
  
   The children of each non-leaf node. Values less than `n_samples` correspond to leaves of the tree which are the original samples. A node `i` greater than or equal to `n_samples` is a non-leaf node and has children `children_[i - n_samples]`. Alternatively at the i-th iteration, children[i][0] and children[i][1] are merged to form node `n_samples + i`.
* **distances_**: ndarray of shape (n_nodes-1,)
  
   Distances between nodes in the corresponding place in `children_`. Only computed if `distance_threshold` is used or `compute_distances` is set to `True`.

## `DBSCAN`

### Parameters of `DBSCAN`

<dl>
<dt><strong>eps</strong>: <span>float, default=0.5</span></dt><dd><p>The maximum distance between two samples for one to be considered
as in the neighborhood of the other. This is not a maximum bound
on the distances of points within a cluster. This is the most
important DBSCAN parameter to choose appropriately for your data set
and distance function.</p>
</dd>
<dt><strong>min_samples</strong>: <span>int, default=5</span></dt><dd><p>The number of samples (or total weight) in a neighborhood for a point
to be considered as a core point. This includes the point itself.</p>
</dd>
<dt><strong>metric</strong>: <span>str, or callable, default=’euclidean’</span></dt><dd><p>The metric to use when calculating distance between instances in a
feature array. If metric is a string or callable, it must be one of
the options allowed by <code>sklearn.metrics.pairwise_distances</code> for
its metric parameter.
If metric is “precomputed”, X is assumed to be a distance matrix and
must be square.
</dd>
<dt><strong>metric_params</strong>: <span>dict, default=None</span></dt><dd><p>Additional keyword arguments for the metric function.</p>
</dd>
<dt><strong>algorithm</strong>: <span>{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’</span></dt><dd><p>The algorithm to be used by the NearestNeighbors module
to compute pointwise distances and find nearest neighbors.
See NearestNeighbors module documentation for details.</p>
</dd>
<dt><strong>leaf_size</strong>: <span>int, default=30</span></dt><dd><p>Leaf size passed to BallTree or cKDTree. This can affect the speed
of the construction and query, as well as the memory required
to store the tree. The optimal value depends
on the nature of the problem.</p>
</dd>
<dt><strong>p</strong>: <span>float, default=None</span></dt><dd><p>The power of the Minkowski metric to be used to calculate distance
between points. If None, then <code><span>p=2</span></code> (equivalent to the Euclidean
distance).</p>
</dd>
</dl>

### Attributes of `DBSCAN`

* **core_sample_indices_**: ndarray of shape (n_core_samples,)

    Indices of core samples.

* **components_**: ndarray of shape (n_core_samples, n_features)

    Copy of each core sample found by training.

* **labels_**: ndarray of shape (n_samples)

    Cluster labels for each point in the dataset given to fit(). Noisy samples are given the label -1.